# Project Name

## Visual Data Science

Clara Pichler, 11917694

### Overview

1. Introduction
- Original Research Questions
- Used Datasets
- Requirements & Dependencies

2. Data Preprocessing
- Loading in the Data Sets
- First Steps
- Missing Values
- Merging Data Sets

3. Data Exploration
- 

4. Conclusion

### Notes

Bis 16.12

__Wrangle:__ 
- Join the two or more datasets you selected into one big data table, and
- Solve issues like formatting issues, missing data, faulty values, and non-matching keys
- _10 points_ extra, if you can (visually!) show and explain the data quality of your dataset (for example, before and after cleaning steps). There are no instructions how such a visualization should look like, since this also highly depends on the chosen domain and data. You will have to come up with your own, creative, solution here.

__Profile:__
- Find at least 3 valuable insights.
- Show the insights by using visualization.

In the Profile part, you will have to explore the data in detail, to completely understand its structure, and to discover any interesting patterns that can be found in there. The following steps are required:

- Find at least 3 informative insights in your dataset, and
- Show and explain the insights using visualizations (one visualization per insight).



## Introduction

### Original Research Questions

- Are there any similarities between countries which win medals often in specific kinds of sports?
- Which factors influence the amount of medals a country wins? 
- Is it possible to predict the number of won medals of a country?
- Are there any trends in the medal distribution over time?
- Which factors play a part in the "under-dog" countries winning suddenly some medals?


### Used Data Sets

__Olympics Medals Data Set__
- _description:_ General information on medalists (Athlete or Team)
- _source:_ https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018
- _note:_ This data set comes with three other data sets that include information on the athletes, hosts and all of the results. I will only focus on the medals so first, second and third place, therefore only using the `olympic_medals.csv` data set. 
- _disadvantages:_ Quote from the description
    - "There are no results for qualification rounds. For instance, event 100-m men contains only final results without semi-finals and other hits."
    - "There is no information about athletes for team competitions that consist of more than 2 participants. Only team records."

__Rugged Data Set__
- _description:_
- _source:_ https://diegopuga.org/data/rugged/ 
- 

__Elevation Extremes Data Set__
- _description:_ A sortable table which lists land surface elevation extremes by country or dependent territory
- _source:_ https://en.wikipedia.org/wiki/List_of_elevation_extremes_by_country 
- 

__World Climate Data__
- _description:_ List of all countries and territories in the world with climate zones and average annual temperatures.
- _source:_ https://weatherandclimate.com/countries 
- 

### Requirements & Dependencies

This project was created using Python 3.12.5. The exact versions of the dependencies can be installed with the following command.

In [157]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [158]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as r
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

## Data Pre-Processing

### Loading in the Data Sets

First and foremost we will load in our data sets. For the rugged data I had some problems with encoding since two countries have special characters in their name, as one can see below. By using `encoding='latin1'` this issue was able to be resolved.

For the elevation data set which I got from Wikipedia, I used `BeautifulSoup` to scrape the table from their website. I treid firts `pd.read_html` however this simple approach did not work for me like it did for others.

In [159]:
with open('data/rugged_data/rugged_data.csv', 'r', encoding='ascii', errors='replace') as f:
    for i, line in enumerate(f):
        if '�' in line:  
            print(f"Problem in row {i + 1}: {line}")

Problem in row 41: "CIV",384,"C�te d'Ivoire",0.224,0.159,0.594,0.069,0.493,31800,7.623,-5.554,26.151,0.000,100.000,0.284,16.359,3239,1575.628,1040.878,1800.013,1352.019,1536.270,-0.860,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,52645.949,4.186,9.457,3.353,4.794,435857,0.000

Problem in row 177: "REU",638,"R�union",3.873,0.962,10.657,1.208,67.425,250,-21.121,55.540,16.400,0.000,58.336,0.008,100,0,,1989.306,3821.115,4588.492,3412.181,,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.000,10.090,0.208,6.256,3.879,0,



In [160]:
df_olympics = pd.read_csv('data/olympics/olympic_medals.csv')
df_rugged = pd.read_csv('data/rugged_data/rugged_data.csv', encoding='latin1')

In [161]:
url = "https://en.wikipedia.org/wiki/List_of_elevation_extremes_by_country"
response = r.get(url)
wiki_page_text = response.text

In [162]:
response.status_code # 200 means everything is A-OK

200

In [163]:
soup = BeautifulSoup(wiki_page_text, 'html.parser')
elevation_table = soup.find('table',{'class':'wikitable sortable'})

headers = [header.text.strip() for header in elevation_table.find_all('th')]

rows = []
for row in elevation_table.find_all('tr')[1:]:  
    cells = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
    rows.append(cells)

df_elevation = pd.DataFrame(rows, columns=headers)

In [164]:
url = "https://weatherandclimate.com/countries"
response = r.get(url)
wiki_page_text = response.text
response.status_code # 200 means everything is A-OK

200

In [165]:
soup = BeautifulSoup(wiki_page_text, 'html.parser')
elevation_table = soup.find('table',{'class':'tb1'})

headers = [header.text.strip() for header in elevation_table.find_all('th')]

rows = []
for row in elevation_table.find_all('tr')[1:]:  
    cells = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
    rows.append(cells)

df_climate = pd.DataFrame(rows, columns=headers)

### First Steps

First we will take a first look at the `df_olympics` dataframe. We will drop unnecessary columns like `athlete_url` and `country_code` since we only need the ISO code which is the `country_3_letter_code` attribute. I do not care about the host country so I changed the `slug_game` attribute to `year` and left out the host name. The type was changed to `int`, all other types are fine. Maybe we will change `medal_type` gold, silver and bronze to 1, 2, and 3 in the future for making it easier to work with. 

I also drop the column `athlete_full_name` which will result in rows that are not unique since for events with the `participant_type` `GameTeam` we have a medal entry for each athlete in the team. Thats why I use `drop_duplicates()`.


In [152]:
df_olympics = df_olympics.drop(['athlete_url', 'country_code', 'athlete_full_name'], axis=1).drop_duplicates()

df_olympics = df_olympics.rename(columns={
    'slug_game': 'year',
    'country_3_letter_code': 'isocode'
})

df_olympics['year'] = df_olympics['year'].str[-4:].astype(int)

df_olympics = df_olympics.reset_index(drop=True)


In [153]:
display(df_olympics.head(5))
display(df_olympics.info())
print(df_olympics['year'].unique())

,discipline_title,year,event_title,event_gender,medal_type,participant_type,participant_title,country_name,isocode
0,Curling,2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,Italy,ITA
1,Curling,2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,Norway,NOR
2,Curling,2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,Sweden,SWE
3,Curling,2022,Women,Women,GOLD,GameTeam,Great Britain,Great Britain,GBR
4,Curling,2022,Women,Women,SILVER,GameTeam,Japan,Japan,JPN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20178 entries, 0 to 20177
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   discipline_title   20178 non-null  object
 1   year               20178 non-null  int64 
 2   event_title        20178 non-null  object
 3   event_gender       20178 non-null  object
 4   medal_type         20178 non-null  object
 5   participant_type   20178 non-null  object
 6   participant_title  5104 non-null   object
 7   country_name       20178 non-null  object
 8   isocode            20178 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.4+ MB


None

[2022 2020 2018 2016 2014 2012 2010 2008 2006 2004 2002 2000 1998 1996
 1994 1992 1988 1984 1980 1976 1972 1968 1964 1960 1956 1952 1948 1936
 1932 1928 1924 1920 1912 1908 1904 1900 1896]


Moving on with `df_elevation`. The attributes `Continent`, `Highest point` and `Lowest point` ist non of my interest from this data set. After renaming the columns, I change all of the values of the attributes 

In [173]:
display(df_elevation.head(5))
display(df_elevation.info())

,Country or region,Continent,Highest point,Maximum elevation,Lowest point,Minimum elevation,Elevation span
0,Afghanistan,Asia,Noshaq,"7492 m24,580 ft",Amu Darya,258 m846 ft,"7234 m23,734 ft"
1,Albania,Europe,Korab,"2764 m9,068 ft",Adriatic Sea,sea level,"2764 m9,068 ft"
2,Algeria,Africa,Mount Tahat,"3003 m9,852 ft",Chott Melrhir,−40 m−131 ft,"3043 m9,984 ft"
3,American Samoa,,Lata Mountain on Ta‘ū,"966 m3,169 ft",South Pacific Ocean,sea level,"966 m3,169 ft"
4,Andorra,Europe,Coma Pedrosa,"2942 m9,652 ft",Gran Valira,"840 m2,756 ft","2102 m6,896 ft"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Country or region  254 non-null    object
 1   Continent          254 non-null    object
 2   Highest point      254 non-null    object
 3   Maximum elevation  254 non-null    object
 4   Lowest point       254 non-null    object
 5   Minimum elevation  254 non-null    object
 6   Elevation span     254 non-null    object
dtypes: object(7)
memory usage: 14.0+ KB


None

In [145]:
df_elevation = df_elevation.drop(['Continent', 'Highest point', 'Lowest point'], axis=1)

df_elevation = df_elevation.rename(columns={
    'Country or region': 'country_name',
    'Maximum elevation': 'max_elevation',
    'Minimum elevation': 'min_elevation',
    'Elevation span' : 'elevation_span'
})

In [146]:
df_elevation = df_elevation.replace("sea level", "0")

for col in ['max_elevation', 'min_elevation', 'elevation_span']: 
    df_elevation[col] = (df_elevation[col].str.split('m')
                         .str[0].str.strip()
                         .str.replace('−', '-', regex=False)
                         .str.replace(',', '.', regex=False))
    df_elevation.replace({col : ''}, np.nan, inplace=True)
    df_elevation[col] = df_elevation[col].astype(float)

In [147]:
display(df_elevation.head(5))


,country_name,max_elevation,min_elevation,elevation_span
0,Afghanistan,7492.0,258.0,7234.0
1,Albania,2764.0,0.0,2764.0
2,Algeria,3003.0,-40.0,3043.0
3,American Samoa,966.0,0.0,966.0
4,Andorra,2942.0,840.0,2102.0


In [176]:
display(df_climate.head(5))
display(df_climate.info())
display(df_climate['Climate zone'].unique(), len(df_climate['Climate zone'].unique()))
display(df_climate[df_climate['Climate zone']=='DSB'])

,Country,Climate zone,Avg °F,Avg °C
0,Afghanistan,DSB,60.26,15.7
1,Albania,CSB,59.31,15.17
2,Algeria,BWH,68.0,20.0
3,American Samoa,AW,82.4,28.0
4,Andorra,CFB,44.91,7.17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country       246 non-null    object
 1   Climate zone  246 non-null    object
 2   Avg °F        246 non-null    object
 3   Avg °C        246 non-null    object
dtypes: object(4)
memory usage: 7.8+ KB


None

array(['DSB', 'CSB', 'BWH', 'AW', 'CFB', 'AF', 'CFA', 'DFB', 'BSK', 'AM',
       'CWB', 'BS', 'BSH', 'DFC', 'CWA', 'ET', 'CFC', 'CSA', 'EF', 'BWK',
       'DWB', 'AS', 'DSC'], dtype=object)

23

,Country,Climate zone,Avg °F,Avg °C
0,Afghanistan,DSB,60.26,15.7


### Missing Values

### Merging Data Sets

## Data Exploration

